In [0]:
%run ../../_utils

In [0]:
from pyspark.sql.functions import count, sum, when

# Camada GOLD

Na camada gold, as limpezas e ajustes já foram feitos, então essa camada é responsável por aplicar regras de negócio, agregações e junções de dados que convirjam para analises.

Essa é uma tabela sumarizada analítica.

O objetivo dela é informar os meses em que tiveram mais pedidos

In [0]:
tb_name = "gold.supermarket_sales_by_month"
target_location = "dbfs:/FileStore/delta/gold/supermarket_sales"

## 1 - Data ingestion


In [0]:
df = spark.sql("select * from silver.supermarket_sales") # outra maneira de ler
# df = spark.read.table("silver.supermarket_sales")


## 2 - preparation


### grouping

In [0]:
df = df.groupBy("sale_year", "sale_month").agg(
    count("invoice_id").alias("total_sales_by_month_year"),
).orderBy("sale_year", "sale_month")

In [0]:
display(df)


## Saving data

In [0]:
save_dataframe(df, format_mode="delta", table_name=tb_name, target_location=target_location)


## create delta table

TODO: implementar UPSERT

o upsert serve para não precisar reescrever todos os dados, mas aproveitar do Delta para fazer um MERGE, caso um registro antigo tenha uma nova versão e INSERT para os dados que são novos

In [0]:
create_table(table_name=tb_name, target_location=target_location)

In [0]:
# exit para fechar a execução
dbutils.notebook.exit("OK")

In [0]:
%sql

select * from gold.supermarket_sales_by_month